In [ ]:
import os
import json

import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role, Session



sess = Session()

role = get_execution_role()
model_dir = "./model" # Replace with S3

In [ ]:
model = PyTorchModel(
    entry_point="inference.py",
    source_dir="src",
    role=role,
    model_data=model_dir,
    framework_version="1.5.0",
    py_version="py3",
)

#### Deploy Model

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer


local_mode = False

if local_mode:
    instance_type = "local"
else:
    instance_type = "ml.c2.xlarge"

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

#### Delete endpoint

In [ ]:
import os

if not local_mode:
    predictor.delete_endpoint()
else:
    os.system("docker container ls | grep 8080 | awk '{print $1}' | xargs docker container rm -f")